In [31]:
import pandas as pd
pd.set_option("display.max_columns", None)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
from scipy import stats
from statsmodels.stats import weightstats as stests
import datetime as dt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV


import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
%matplotlib inline
import os
from sqlalchemy import create_engine
import time
from datetime import datetime, date, time
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings('ignore')
from PIL import Image
import scipy.stats as st

from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

# Classifier Libraries
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.naive_bayes import GaussianNB

from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import confusion_matrix, classification_report, plot_confusion_matrix
from sklearn.metrics import log_loss, roc_auc_score, recall_score, precision_score, average_precision_score, f1_score, classification_report, accuracy_score, plot_roc_curve, plot_precision_recall_curve, plot_confusion_matrix
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [7]:
df = pd.read_csv('df_modeling_sample.csv')
df.columns

Index(['Unnamed: 0', 'distance', 'weather_delay_mean', 'weather_delay_mean.1',
       'nas_delay_mean', 'security_delay_mean', 'late_aircraft_delay_mean',
       'arr_delay', 'flight_delay', 'flight_time_afternoon',
       'flight_time_early-morning', 'flight_time_evening',
       'flight_time_mid-night', 'flight_time_morning', 'flight_time_night',
       'flight_season_spring', 'flight_season_summer', 'flight_season_winter',
       'flight_length_medium', 'flight_length_short'],
      dtype='object')

In [8]:
df.drop(axis = 1, labels = ['Unnamed: 0', 'flight_delay', 'weather_delay_mean.1'], inplace = True)

In [9]:
y = df['arr_delay']

In [10]:
X = df.loc[:, df.columns != 'arr_delay']

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.80)

In [12]:
scaler = StandardScaler()
scaled_X_train = scaler.fit_transform(X_train)
scaled_X_test = scaler.transform(X_test)

In [13]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

In [14]:
xg_reg.fit(scaled_X_train, y_train)

[14:23:34] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.3, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.1, max_delta_step=0,
             max_depth=5, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=10, n_jobs=8,
             num_parallel_tree=1, objective='reg:linear', predictor='auto',
             random_state=0, reg_alpha=10, reg_lambda=1, scale_pos_weight=1,
             subsample=1, tree_method='exact', validate_parameters=1,
             verbosity=None)

In [15]:
preds = xg_reg.predict(X_test)

In [18]:
mse = mean_squared_error(y_test, preds)
print('MSE: %f' %(mse))

MSE: 917.987429


In [20]:
rmse = np.sqrt(mean_squared_error(y_test, preds))
print('RMSE: %f' %(rmse))

RMSE: 30.298307


In [22]:
r2_score_xgb = r2_score(y_test, preds)
print(r2_score_xgb)

0.018366481867443496


#### Grid Search

In [25]:
param_grid_XG = {
    'objective': ['reg:squarederror'],
    'colsample_by_tree': [0.4, 0.5, 0.6],
    'learning_rate': [0.001, 0.01, 0.1, 0.5, 1],
    'max_depth': [3, 5, 7, 10, 15],
    'alpha': [1, 2, 3, 5, 10],
    'n_estimators': [5, 10, 30, 50]}

In [26]:
xg_reg_gs = xgb.XGBRegressor()

In [27]:
grid_XG = GridSearchCV(estimator=xg_reg_gs, param_grid=param_grid_XG, scoring='r2', cv=5, verbose=1, n_jobs=-1)

In [28]:
grid_XG.fit(X_train, y_train)

Fitting 5 folds for each of 1500 candidates, totalling 7500 fits
[14:58:09] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "colsample_by_tree" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    enable_categorical=False, gamma=None,
                                    gpu_id=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, max_delta_step=None,
                                    max_depth=None, min_child_weight=None,
                                    missing=nan, monotone_constraints=None,
                                    n...
                                    random_state=None, reg_alpha=None,
                                    reg_lambda=None, scale_pos_weight=None,
                                    subsample=None, tree_method=None,
                                    val

In [29]:
y_pred_XG = grid_XG.predict(X_test)

In [32]:
mae_xg = mean_absolute_error(y_test, y_pred_XG)
mse_xg = np.sqrt(mean_squared_error(y_test, y_pred_XG))
r2_xg =metrics.r2_score(y_test, y_pred_XG)

In [33]:
print('Mean Absolute Error: ', mae_xg)
print('Mean Square Error: ', mse_xg)
print('R2 Value: ', r2_xg)

Mean Absolute Error:  19.302723665721714
Mean Square Error:  29.85465356019382
R2 Value:  0.046903845996042026
